# Causal Analysis Guide for WiDS Workshop

Hello and welcome to the **WiDS Causal Analysis Workshop**! Before we begin, make sure you have set up your environment as per the instructions in our `README.md` file on GitHub.

This Jupyter Notebook is your one-stop guide to Causal Analysis, starting from **Causal Discovery** to **Causal Inference**. We’ll walk you through the entire process, which includes loading your dataset, training the causal discovery model (using [DECI](https://github.com/microsoft/causica/tree/main)/[DirectLiNGAM](https://lingam.readthedocs.io/en/latest/tutorial/lingam.html)), generating edge confidence for the causal graph, estimating errors for the fitted model, and finally, estimating Average Treatment Effect (ATE), Conditional Average Treatment Effect (CATE) and Individual Treatment Effect (ITE) using [DECI](https://github.com/microsoft/causica/tree/main)/[DML](https://econml.azurewebsites.net/spec/estimation/dml.html).

Note: This notebook is adapted from [causica example notebook](https://github.com/microsoft/causica/blob/main/examples/multi_investment_sales_attribution.ipynb).

By the end of this guide, you’ll have a solid understanding of causal analysis and be equipped to perform causal discovery and inference analysis on your own datasets. So, let’s get started!

# Environment Setup
If you have not already set up your environment, please follow the instructions in `README.md` before proceeding. 

You can also download and install Graphviz on your system from https://graphviz.org/ and `pip install graphviz`  for improved visualisations of the causal graphs.

In [ ]:
from operator import itemgetter
import warnings
import os

import fsspec
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch


from pytorch_lightning.callbacks import TQDMProgressBar
from tensordict import TensorDict

from causica.datasets.causica_dataset_format import CAUSICA_DATASETS_PATH, Variable
from causica.distributions import ContinuousNoiseDist
from causica.lightning.data_modules.basic_data_module import BasicDECIDataModule
from causica.lightning.modules.deci_module import DECIModule
from causica.sem.sem_distribution import SEMDistributionModule
from causica.sem.structural_equation_model import ite
from causica.training.auglag import AugLagLRConfig

warnings.filterwarnings("ignore")
test_run = bool(os.environ.get("TEST_RUN", False))  # used by testing to run the notebook as a script
DATADOWNLOAD = True
DATA_PATH = './data/multi_attribution_data_20220819_data.csv' # from https://azuastoragepublic.z6.web.core.windows.net/causal_ai_suite/multi_attribution_data_20220819_data.csv
VARIABLES_PATH = './data/multi_attribution_data_20220819_data_types.json' # from https://azuastoragepublic.z6.web.core.windows.net/causal_ai_suite/multi_attribution_data_20220819_data_types.json
BATCH_SIZE = 128

# Dataset

### Example Dataset Overview: Multi-investment Attribution: Distinguish the Effects of Multiple Outreach Efforts

<img src="https://www.microsoft.com/en-us/research/uploads/prod/2020/05/Attribution.png" width="250">

A software company would like to know whether its multiple outreach efforts to their business customers are successful in boosting sales. They would also like to learn how to better target different incentives to different customers. In other words, they would like to learn the **treatment effect** of each investment on customers' total expenditure on the company’s products: particularly the **heterogeneous treatment effect**. 

In an ideal world, the company would run experiments where each customer would receive a random assortment of investments. However, this approach can be logistically prohibitive or strategically unsound: the company might not have the resources to design such experiments or they might not want to risk major revenue losses by randomizing their incentives to top customers.

In this notebook, we show how tools from the [Causica](https://github.com/microsoft/causica), library can use historical investment data to learn the effects of multiple investments.

For this exercise, we create simulated data that recreates some key characteristics of real data from a software company facing this type of decision. Simulating data protects the company’s privacy. Because we create the data, we are also in the unusual position of knowing the true causal graph and true effects of each investments, so we can compare the results of our estimation to this ground truth.

In the next section, we introduce this simulated data. We then discover the causal graph, the relationship between each variable in the simulated data. We use this generated graph, to estimate the treatment effects. 

### Data

The simulated dataset contains 10,000 customers. 

We create one outcome of interest:

Feature Name | Type | Details 
:--- |:--- |:--- 
**Revenue** | continuous | \$ Annual revenue from customer given by the amount of software purchased

We consider three possible treatments, the interventions whose impact we wish to measure:

Feature Name | Type | Details 
:--- |:--- |:--- 
**Tech Support** | binary | whether the customer received tech support during the year
**Discount** | binary | whether the customer was given a discount during the year
**New Strategy** | binary | whether the customer was targeted for a new engagement strategy with different outreach behaviors

Finally, we consider a variety of additional customer characteristics that may affect revenue. Including these types of features is crucial for causal analysis in order to map the full causal graph and separate the true effects of treatments on outcomes from other correlation generated by other influences. 

Feature Name | Type | Details 
:--- |:--- |:--- 
**Global Flag** | binary | whether the customer has global offices
**Major Flag** | binary | whether the customer is a large consumer in their industry (as opposed to SMC - Small Medium Corporation - or SMB - Small Medium Business)
**SMC Flag** | binary | whether the customer is a Small Medium Corporation (SMC, as opposed to major and SMB)
**Commercial Flag** | binary | whether the customer's business is commercial (as opposed to public sector)
**Planning Summit** | binary | whether a sales team member held an outreach event with the customer during the year
**New Product Adoption** | binary | whether the customer signed a contract for any new products during the year
**IT Spend** | continuous | \$ spent on IT-related purchases 
**Employee Count** | continuous | number of employees
**PC Count** | continuous | number of PCs used by the customer
**Size** | continuous | customer's total revenue in the previous calendar year

In simulating the data, we maintain some key characteristics of the data from the real company example, including some correlation patterns between features and some potentially difficult data characteristics, such as large outliers.

#### Import the Data

In [ ]:
df = pd.read_csv(DATA_PATH)
with fsspec.open(VARIABLES_PATH, mode="r", encoding="utf-8") as f:
    variables_spec = json.load(f)["variables"]

data_module = BasicDECIDataModule(
    df.loc[:, "Global Flag":"Revenue"], # remove ground truth from dataframe for simulated data
    variables=[Variable.from_dict(d) for d in variables_spec],
    batch_size=BATCH_SIZE,
    normalize=True,
)    
num_nodes = len(data_module.dataset_train.keys())

In [ ]:
# Data sample
print("Data Shape:", df.shape)
df.head()

# Let’s construct the Causal Graph

You should have a sheet of paper in front of you, with all the variables from the dataset represented as nodes. Currently, this graph has no edges.

<img src="./img/DrawCausalGraph.png" width="800">

 Your task is to draw edges between the nodes (variables) that you believe have a causal relationship. Remember, the direction of the edge is important, so make sure to indicate it. Please don't draw a fully connected graph :)

 **Please ensure your graph does not have cycles**.

# Discover the Causal Graph using DECI

### [Optional] Adding prior domain knowledge
#### Note : Provided constraints are for Simulated data. You will need to create appropriate constraints as required for the real data
To improve the quality of the learned graph, it is possible to place constraints on the graph that DECI learns. The constraints come in two flavours:
 - *negative constraints* mean a certain edge cannot exist in the graph,
 - *positive constraints* mean a certain edge must exist in the graph.

In [ ]:
# The constraint matrix has the same shape as the adjacency matrix
# A `nan` value means no constraint
# A 0 value means a negative constraint
# A 1 value means a positive constraint
node_name_to_idx = {key: i for i, key in enumerate(data_module.dataset_train.keys())}
constraint_matrix = np.full((num_nodes, num_nodes), np.nan, dtype=np.float32)

First, we introduce the constraint that Revenue cannot be the cause of any other node, except possibly Planning Summit.

In [ ]:
revenue_idx = node_name_to_idx["Revenue"]
planning_summit_idx = node_name_to_idx["Planning Summit"]
constraint_matrix[revenue_idx, :] = 0.0
constraint_matrix[revenue_idx, planning_summit_idx] = np.nan

Second, we say that certain basic attributes of companies cannot be changed by other variables (at least on the time scale we are considering). The attributes we constraint to have no parents are: Commerical Flag, Major Flag, SMC Flag, PC Count, Employee Count, Global Flag, Size.

In [ ]:
non_child_nodes = [
    "Commercial Flag",
    "Major Flag",
    "SMC Flag",
    "PC Count",
    "Employee Count",
    "Global Flag",
    "Size",
]
non_child_idxs = itemgetter(*non_child_nodes)(node_name_to_idx)
constraint_matrix[:, non_child_idxs] = 0.0

Finally, we make a constraint that says that different engagements do not directly cause one another. For example, giving Tech Support to a company is not a valid reason to give / not give them a Discount.

In [ ]:
engagement_nodes = ["Tech Support", "Discount", "New Engagement Strategy"]
engagement_idxs = itemgetter(*engagement_nodes)(node_name_to_idx)
for i in engagement_idxs:
    constraint_matrix[engagement_idxs, i] = 0.0

### Creating and training DECI model

The following snippets step through the process of:
 - configuring and creating a DECI model
 - training a model with graph constraints

#### DECI configuration

The DECI model has a number of hyperparameters, but attention need not be paid to all of them. Here we highlight key hyperparameters that might be changed to improve performance:
 - `noise_dist` is the type of DECI model that is trained with. It should be either Gaussian or Spline. Use a Spline for highly non-Gaussian data, or to fit a better density model of the observational data.
 
Other hyperparameters are less frequently changed.

In [ ]:
pl.seed_everything(seed=4)  # set the random seed

lightning_module = DECIModule(
    noise_dist=ContinuousNoiseDist.GAUSSIAN,
    prior_sparsity_lambda=43.0,
    init_rho=30.0,
    init_alpha=0.20,
    auglag_config=AugLagLRConfig(
        max_inner_steps=3400,
        max_outer_steps=8,
        lr_init_dict={
            "icgnn": 0.00076,
            "vardist": 0.0098,
            "functional_relationships": 3e-4,
            "noise_dist": 0.0070,
        },
    ),
)

lightning_module.constraint_matrix = torch.tensor(constraint_matrix)

trainer = pl.Trainer(
    accelerator="auto",
    max_epochs=2000,
    fast_dev_run=test_run,
    callbacks=[TQDMProgressBar(refresh_rate=19)],
    enable_checkpointing=False,
)

In [ ]:
trainer.fit(lightning_module, datamodule=data_module)

### Saving and Loading a DECI model

In [ ]:
torch.save(lightning_module.sem_module, "deci.pt")

In [ ]:
sem_module: SEMDistributionModule = torch.load("deci.pt")

# create a structural equation model using the most likely graph
sem = sem_module().mode

### Visualizing Causal Graph using "Mode" of the posterior distribution

In [ ]:
graph = nx.from_numpy_array(sem.graph.cpu().numpy(), create_using=nx.DiGraph)
graph = nx.relabel_nodes(graph, dict(enumerate(data_module.dataset_train.keys())))

fig, axis = plt.subplots(1, 1, figsize=(8, 8))

labels = {node: i for i, node in enumerate(graph.nodes)}

# layout = nx.nx_agraph.graphviz_layout(graph, prog="dot")
layout = nx.circular_layout(graph)

for node, i in labels.items():
    axis.scatter(layout[node][0], layout[node][1], label=f"{i}: {node}")
axis.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

nx.draw_networkx(graph, pos=layout, with_labels=True, arrows=True, labels=labels, ax=axis)

### Compare to Ground Truth Causal Graph

In [ ]:
adjacency_path = "./data/true_graph_gml_string.txt"
with fsspec.open(adjacency_path, mode="r", encoding="utf-8") as f:
    true_adj = nx.parse_gml(f.read())

fig, axis = plt.subplots(1, 1, figsize=(8, 8))

# try:
#     layout = nx.nx_agraph.graphviz_layout(true_adj, prog="dot")
# except (ModuleNotFoundError, ImportError):
#     layout = nx.circular_layout(graph)

for node, i in labels.items():
    axis.scatter(layout[node][0], layout[node][1], label=f"{i}: {node}")
axis.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

nx.draw_networkx(true_adj, pos=layout, with_labels=True, arrows=True, labels=labels, ax=axis)

### Concluding graph discovery
The DECI model from Causica offers us a way to discover a causal graph from observational data. The learned graph can be iteratively refined by adding new graph constraints and retraining DECI to obtain a more realistic graph. 

# (Alternative) - Discover the Causal Graph using DirectLiNGAM 
In case you have **trouble installing DECI**, we can use DirectLiNGAM method which also accepts prior domain constraints.

Note: If you **ran DECI successfully**, then please **skip this session** and go ahead to the Treatment Effect Estimation section

### Adding prior domain knowledge

In [ ]:
import graphviz
import lingam
from lingam.utils import make_dot

print([np.__version__, pd.__version__, graphviz.__version__, lingam.__version__])

np.set_printoptions(precision=3, suppress=True)
np.random.seed(100)

In [ ]:
'''

0: No direct path from variable i to variable j

1: Directed path exists from variable i and variable j

-1: No prior knowledge is available to know if either of the two cases above (0 or 1) is true.

'''
node_name_to_idx = {key: i for i, key in enumerate(df.columns)}
prior_knowledge = -1 * np.ones((len(df.loc[:, "Global Flag":"Revenue"].columns), len(df.loc[:, "Global Flag":"Revenue"].columns)))

First, we introduce the constraint that Revenue cannot be the cause of any other node, except possibly Planning Summit.

In [ ]:
revenue_idx = node_name_to_idx["Revenue"]
planning_summit_idx = node_name_to_idx["Planning Summit"]
prior_knowledge[:, revenue_idx] = 0.0
prior_knowledge[planning_summit_idx, revenue_idx] = -1

Second, we say that certain basic attributes of companies cannot be changed by other variables (at least on the time scale we are considering). The attributes we constraint to have no parents are: Commerical Flag, Major Flag, SMC Flag, PC Count, Employee Count, Global Flag, Size.

In [ ]:
non_child_nodes = [
    "Commercial Flag",
    "Major Flag",
    "SMC Flag",
    "PC Count",
    "Employee Count",
    "Global Flag",
    "Size",
]
non_child_idxs = itemgetter(*non_child_nodes)(node_name_to_idx)
prior_knowledge[non_child_idxs, :] = 0.0

Finally, we make a constraint that says that different engagements do not directly cause one another. For example, giving Tech Support to a company is not a valid reason to give / not give them a Discount.

In [ ]:
engagement_nodes = ["Tech Support", "Discount", "New Engagement Strategy"]
engagement_idxs = itemgetter(*engagement_nodes)(node_name_to_idx)
for i in engagement_idxs:
    prior_knowledge[i, engagement_idxs] = 0.0

### Learn the Causal Graph

In [ ]:
model = lingam.DirectLiNGAM(prior_knowledge=prior_knowledge)
model.fit(df.loc[:, "Global Flag":"Revenue"])

### Visualize the Causal Graph

In [ ]:
adj_matrix = model.adjacency_matrix_.copy().T
graph = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)
graph = nx.relabel_nodes(graph, dict(enumerate(df.columns)))

fig, axis = plt.subplots(1, 1, figsize=(8, 8))

labels = {node: i for i, node in enumerate(graph.nodes)}
layout = nx.circular_layout(graph)

for node, i in labels.items():
    axis.scatter(layout[node][0], layout[node][1], label=f"{i}: {node}")
axis.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

nx.draw_networkx(graph, pos=layout, with_labels=True, arrows=True, labels=labels, ax=axis)

### Compare to Ground Truth Causal Graph

In [ ]:
adjacency_path = "./data/true_graph_gml_string.txt"
with fsspec.open(adjacency_path, mode="r", encoding="utf-8") as f:
    true_adj = nx.parse_gml(f.read())

fig, axis = plt.subplots(1, 1, figsize=(8, 8))

# try:
#     layout = nx.nx_agraph.graphviz_layout(true_adj, prog="dot")
# except (ModuleNotFoundError, ImportError):
#     layout = nx.circular_layout(graph)

for node, i in labels.items():
    axis.scatter(layout[node][0], layout[node][1], label=f"{i}: {node}")
axis.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

nx.draw_networkx(true_adj, pos=layout, with_labels=True, arrows=True, labels=labels, ax=axis)

# Did you get the edges right?

# Treatment effect estimation

The causal graph identifies the likely paths of connection between features. This next step will quantify the strength of those relationships between treatment and outcomes. Our tools estimate both the average treatment effect across all customers (ATE) and how these treatment effects vary across customer features (heterogeneous treatment effect, HTE). If we are confident that we have included every feature that affects treatment intensity, we can also interpret these estimates as individual treatment effects (ITE), or the effect of treating a specific customer.

We will use two different methods for estimation (DECI and Double ML) and compare how well they do.

#### Import the Ground Truth Causal Effects

Unlike most real-world causal use cases, in this case we know the true causal relationships between treatments and outcome because we have simulated the data.

We also extract those effects, created during data generation, to check our later causal effect estimates.

In [ ]:
outcome = "Revenue"
treatment_columns = ["Tech Support", "Discount", "New Engagement Strategy"]

# extract ground truth effects
ground_truth_effects = df.loc[:, "Direct Treatment Effect: Tech Support":]

ground_truth_ites = {
    treatment: ground_truth_effects[f"Total Treatment Effect: {treatment}"] for treatment in treatment_columns
}

ground_truth_ates = {key: val.mean(axis=0) for key, val in ground_truth_ites.items()}

# Question: Of the three treatments, which do you think is most effective?

## Average Treatment Effect using DECI (ATE)
We can estimate the average treatment and estimate the error using samples from the intervened SEM.

In [ ]:
outcome = "Revenue"
treatment_columns = ["Tech Support", "Discount", "New Engagement Strategy"]

In [ ]:
revenue_estimated_ate = {}
num_samples = 20000
sample_shape = torch.Size([num_samples])
normalizer = data_module.normalizer

# extract estimated ATEs from DECI
for treatment in treatment_columns:
    intervention_a = TensorDict({treatment: torch.tensor([1.0])}, batch_size=tuple())
    intervention_b = TensorDict({treatment: torch.tensor([0.0])}, batch_size=tuple())

    rev_a_samples = normalizer.inv(sem.do(interventions=intervention_a).sample(sample_shape))[outcome]
    rev_b_samples = normalizer.inv(sem.do(interventions=intervention_b).sample(sample_shape))[outcome]

    ate_mean = rev_a_samples.mean(0) - rev_b_samples.mean(0)
    ate_std = np.sqrt((rev_a_samples.var(0) + rev_b_samples.var(0)) / num_samples)

    revenue_estimated_ate[treatment] = (
        ate_mean.cpu().numpy()[0],
        ate_std.cpu().numpy()[0],
    )
revenue_estimated_ate

Let's compare the estimated average treatment effects of DECI with the ground truth.

In [ ]:
fig, axes = plt.subplots(1, len(ground_truth_ates), figsize=(16, 4))
fig.suptitle("Comparison of Ground Truth ATEs with DECI estimates")
for ax, treatment in zip(axes, revenue_estimated_ate.keys()):
    ax.errorbar([0], [revenue_estimated_ate[treatment][0]], [2 * revenue_estimated_ate[treatment][1]], fmt="o")
    ax.scatter([0], [ground_truth_ates[treatment]], color="r", label="Ground Truth")
    ax.legend()
    ax.set_title(treatment)
    ax.set_ylabel("Estimated ATE on Revenue ($)")

Question: How well did DECI estimate the ATEs?

## Average Treatment Effect using DML (ATE)

In [ ]:
from econml.dml import LinearDML
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

Using Double ML, let's estimate the ATE for each intervention and compare to the ground truth and DECI estimates.

In [ ]:
revenue_estimated_ate_econml = {}

# prepare treatment, outcome, and confounder data
Y = df["Revenue"]  # outcome
W = df[["SMC Flag", "Major Flag", "Commercial Flag", "Employee Count", "PC Count", "IT Spend"]] # confounders

# set first-stage models
outcome_model = RandomForestRegressor(random_state=100, max_depth=5) # predict outcome from confounders
treatment_model = RandomForestClassifier(random_state=100, max_depth=5) # predict treatment from confounders

# run DML for each treatment
for treatment in treatment_columns:
    T = df[treatment]  # intervention

    est = LinearDML(model_y=outcome_model,
                model_t=treatment_model,
                discrete_treatment=True, # specify that treatment should be treated as categorical
                categories=[0,1], # specify the category labels of the treatment
                random_state=123) # set seed for reproducibility
    est.fit(Y=Y, T=T, W=W, X=None)
    ate = est.effect_inference().point_estimate
    ate_se = est.effect_inference().stderr

    # save output
    revenue_estimated_ate_econml[treatment] = (
    ate,
    ate_se
    )

revenue_estimated_ate_econml

**Let's look at the output for New Engagement Strategy and interpret the results**

In [ ]:
est.summary(alpha=0.05)

**Question: The point estimate is the estimated ATE of New Engagement Strategy on Revenue and the summary table includes inference results.**

**Do we have evidence that New Engagement Strategy is effective in moving Revenue?**

#### Now, let's compare the estimated ATE of each treatment to the ground truth.

In [ ]:
fig, axes = plt.subplots(1, len(ground_truth_ates), figsize=(16, 4))
fig.suptitle("Comparison of Ground Truth ATEs with DECI & DML estimates")
for ax, treatment in zip(axes, revenue_estimated_ate_econml.keys()):
    ax.errorbar([0], [revenue_estimated_ate[treatment][0]], [2 * revenue_estimated_ate[treatment][1]], fmt="o")
    ax.scatter([1], [ground_truth_ates[treatment]], color="r", label="Ground Truth")
    ax.errorbar([2], [revenue_estimated_ate_econml[treatment][0]], [2 * revenue_estimated_ate_econml[treatment][1]], fmt="o")
    ax.set_xticks(np.arange(3))
    ax.set_xticklabels(['DECI'] + ['GroundTruth'] + ['DML'])
    ax.legend()
    ax.set_title(treatment)
    ax.set_ylabel("Estimated ATE on Revenue ($)")

**Question: How well did Double ML estimate the ATEs compared to DECI?**

## Exercise: Try adding/removing confounders. How does the estimated ATE change?
You can try with the confounders from your DECI/DirectLiNGAM output graph and,

you can also try with your own set of confounders from your drawing sheet!

In [ ]:
revenue_estimated_ate_confounderchanged = {}

# prepare treatment, outcome, and confounder data
Y = df["Revenue"]  # outcome
# W = df[[]] #  -----------------------------------------------> Add your confounders in the list e.g df[["SMC Flag", "Major Flag"]]

# set first-stage models
outcome_model = RandomForestRegressor(random_state=100, max_depth=5) # predict outcome from confounders
treatment_model = RandomForestClassifier(random_state=100, max_depth=5) # predict treatment from confounders

# run DML for each treatment
for treatment in treatment_columns:
    T = df[treatment]  # intervention

    est = LinearDML(model_y=outcome_model,
                model_t=treatment_model,
                discrete_treatment=True, # specify that treatment should be treated as categorical
                categories=[0,1], # specify the category labels of the treatment
                random_state=123) # set seed for reproducibility
    est.fit(Y=Y, T=T, W=W, X=None)
    ate = est.effect_inference().point_estimate
    ate_se = est.effect_inference().stderr

    # save output
    revenue_estimated_ate_confounderchanged[treatment] = (
    ate,
    ate_se
    )

revenue_estimated_ate_confounderchanged

In [ ]:
fig, axes = plt.subplots(1, len(ground_truth_ates), figsize=(18, 7))
fig.suptitle("Comparison of Ground Truth ATEs with DECI & DML estimates")
for ax, treatment in zip(axes, revenue_estimated_ate_econml.keys()):
    ax.scatter([0], [ground_truth_ates[treatment]], color="r", label="Ground Truth")
    ax.errorbar([1], [revenue_estimated_ate[treatment][0]], [2 * revenue_estimated_ate[treatment][1]], fmt="o")
    ax.errorbar([2], [revenue_estimated_ate_econml[treatment][0]], [2 * revenue_estimated_ate_econml[treatment][1]], fmt="o")
    # ax.errorbar([3], [revenue_estimated_ate_confounderchanged[treatment][0]], [2 * revenue_estimated_ate_confounderchanged[treatment][1]], fmt="o") #-----------------------> uncomment
    # ax.set_xticks(np.arange(4)) #-----------------------> uncomment
    # ax.set_xticklabels(['GroundTruth'] + ['DECI'] + ['DML RF True Confounder'] + ['DML RF Modified Confounder'], rotation=90) #-----------------------> uncomment
    ax.set_title(treatment)
    ax.set_ylabel("Estimated ATE on Revenue ($)")
    ax.grid()
    
plt.tight_layout()


## Exercise: Try using different first-stage ML models
Note : You can use any sklearn model.
You can also use LightGBM model and import it as `from lightgbm import LGBMClassifier, LGBMRegressor`

**How does the estimated ATE change?**

In [ ]:
# from sklearn.linear_model import LogisticRegression, LinearRegression # -----------------------> uncomment

revenue_estimated_ate_lr = {}

# prepare treatment, outcome, and confounder data
T = df['Discount']
Y = df["Revenue"]  # outcome
W = df[["SMC Flag", "Major Flag", "Commercial Flag", "Employee Count", "PC Count", "IT Spend"]] # confounders

# set first-stage models
# outcome_model = LinearRegression() # predict outcome from confounders -----------------------> uncomment
# treatment_model = LogisticRegression() # predict treatment from confounders -----------------------> uncomment

# run DML for each treatment
for treatment in treatment_columns:
    T = df[treatment]  # intervention

    est = LinearDML(model_y=outcome_model,
                model_t=treatment_model,
                discrete_treatment=True, # specify that treatment should be treated as categorical
                categories=[0,1], # specify the category labels of the treatment
                random_state=123) # set seed for reproducibility
    est.fit(Y=Y, T=T, W=W, X=None)
    ate = est.effect_inference().point_estimate
    ate_se = est.effect_inference().stderr

    # save output
    revenue_estimated_ate_lr[treatment] = (
    ate,
    ate_se
    )

revenue_estimated_ate_lr

In [ ]:
fig, axes = plt.subplots(1, len(ground_truth_ates), figsize=(18, 7))
fig.suptitle("Comparison of Ground Truth ATEs with DECI & DML estimates")
for ax, treatment in zip(axes, revenue_estimated_ate_econml.keys()):
    ax.scatter([0], [ground_truth_ates[treatment]], color="r", label="Ground Truth")
    ax.errorbar([1], [revenue_estimated_ate[treatment][0]], [2 * revenue_estimated_ate[treatment][1]], fmt="o")
    ax.errorbar([2], [revenue_estimated_ate_econml[treatment][0]], [2 * revenue_estimated_ate_econml[treatment][1]], fmt="o")
    ax.errorbar([3], [revenue_estimated_ate_confounderchanged[treatment][0]], [2 * revenue_estimated_ate_confounderchanged[treatment][1]], fmt="o") 
    # ax.errorbar([4], [revenue_estimated_ate_lr[treatment][0]], [2 * revenue_estimated_ate_lr[treatment][1]], fmt="o") # -----------------------> uncomment
    # ax.set_xticks(np.arange(5)) # -----------------------> uncomment
    # ax.set_xticklabels(['GroundTruth'] + ['DECI'] + ['DML RF True Confounder'] + ['DML RF Modified Confounder'] + ['DML LR True Confounder'], rotation=90) # -----------------------> uncomment
    ax.set_title(treatment)
    ax.set_ylabel("Estimated ATE on Revenue ($)")
    ax.grid()
plt.tight_layout()

### Estimating heterogeneous treatment effect (HTE) with DML

Suppose the effect of the discount is more effective among certain types of businesses. Knowing this would allow us to optimize which customers should receive which treatments. Let's estimate the treatment effect of Discount for different customer sizes. Here, 'Size' is called an effect modifier, as it modifies the effectiveness of the treatment.

In [ ]:
T = df["Tech Support"]  # intervention, treatment
Y = df["Revenue"]  # amount of product purchased, outcome
W = df[["SMC Flag", "Major Flag", "Commercial Flag", "Employee Count", "PC Count", "IT Spend"]]
X = df[["Size"]] # effect modifier

In [ ]:
outcome_model = RandomForestRegressor(random_state=100, max_depth=5) # predict outcome from confounders
treatment_model = RandomForestClassifier(random_state=100, max_depth=5) # predict treatment from confounders

est = LinearDML(model_y=outcome_model,
                model_t=treatment_model,
                discrete_treatment=True, # specify that treatment should be treated as categorical
                categories=[0,1], # specify the categories of the treatment
                random_state=123) # set seed for reproducibility
est.fit(Y=Y, T=T, W=W, X=X)
est.summary(alpha=0.05)

In [ ]:
print("The estimated effect of tech support (conditioned on company size) is:", est.intercept_, "+", est.coef_[0], "*size")

#### Interpretation
If a company has 0 previous year revenue (Size=0), then offering tech support to them will increase our revenue through software purchases by ~$6924 .

For every `$1` increase in the company's previous year Revenue (Size), we expect that offering tech support will further increase their software purchases by ~$0.02.

e.g. for a company of size 1 million, offering tech support will increase our revenue by ~$6924 + 0.02 * 1 million.

=> Tech Support is more effective for companies with larger previous year revenue (indicaed by variable size)

### (Optional) Estimating individual treatment effect (ITE) with DECI

In [ ]:
revenue_estimated_ite = {}

base_noise = sem.sample_to_noise(data_module.dataset_train)

for treatment in treatment_columns:
    do_sem = sem.do(interventions=TensorDict({treatment: torch.tensor([1.0])}, batch_size=tuple()))
    do_a_cfs = normalizer.inv(do_sem.noise_to_sample(base_noise))[outcome].cpu().detach().numpy()[:, 0]
    do_sem = sem.do(interventions=TensorDict({treatment: torch.tensor([0.0])}, batch_size=tuple()))
    do_b_cfs = normalizer.inv(do_sem.noise_to_sample(base_noise))[outcome].cpu().detach().numpy()[:, 0]
    revenue_estimated_ite[treatment] = do_a_cfs - do_b_cfs

revenue_estimated_ite

In [ ]:
fig, axes = plt.subplots(1, len(ground_truth_ates), figsize=(16, 4))
fig.suptitle("Comparison of Ground Truth ITEs with DECI estimates")
for ax, treatment in zip(axes, revenue_estimated_ite.keys()):
    ax.scatter(revenue_estimated_ite[treatment], ground_truth_ites[treatment])
    ax.set_title(treatment)
    ax.set_xlabel("Estimated ITE ($)")
    ax.set_ylabel("Ground Truth ITE ($)")

## (Optional) Advanced Evaluations for DECI

### Estimating Edge Confidence estimate using samples from posterior distribution

In [ ]:
sample_size = 100
adj_matrix_sum = np.zeros((num_nodes, num_nodes))

for i in range(sample_size):

    # Sample a graph from the posterior distribution
    adj_matrix_sum += sem_module()._adjacency_dist.sample().numpy()

# modify adj_matrix_sum such that it's a DAG. adj_matrix_sum should not have [x,y]>0 and [y,x]>0 at the same time for any x and y. If it does, keep the maximum value and set the other value to 0.
for i in range(num_nodes):
    for j in range(num_nodes):
        if adj_matrix_sum[i][j] > adj_matrix_sum[j][i]:
            adj_matrix_sum[j][i] = 0
        else:
            adj_matrix_sum[i][j] = 0

adj_matrix_avg = adj_matrix_sum / sample_size

print(adj_matrix_sum)
print(adj_matrix_avg)

In [ ]:
# plot the average adjacency matrix
adj_matrix_avg[adj_matrix_avg < 0.5] = 0 # Only keep edges with weight >= 0.5

graph = nx.from_numpy_array(adj_matrix_avg, create_using=nx.DiGraph)
graph = nx.relabel_nodes(graph, dict(enumerate(data_module.dataset_train.keys())))

labels = {node: i for i, node in enumerate(graph.nodes)}

layout = nx.circular_layout(graph)
# layout = nx.nx_agraph.graphviz_layout(graph, prog="dot")

fig, axis = plt.subplots(1, 1, figsize=(8, 8))

for node, i in labels.items():
    axis.scatter(layout[node][0], layout[node][1], label=f"{i}: {node}")

# add legend to right side of the plot outside the plot area
axis.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

nx.draw_networkx(graph, pos=layout, with_labels=True, arrows=True, labels=labels, ax=axis)  

# Get weights of each edge and assign to labels
labels = nx.get_edge_attributes(graph, "weight")

# Draw edge labels using layout and list of labels
nx.draw_networkx_edge_labels(graph, pos=layout, edge_labels=labels)

### Estimate Edge Confidence from DECI

In [ ]:
def num_lower_tri_elements_to_n(x: int) -> int:
    """
    Calculate the size of the matrix from the number of strictly lower triangular elements.

    We have x = n(n - 1) / 2 for some n
    n² - n - 2x = 0
    so n = (1 + √(1 + 8x)) / 2
    """
    val = int(np.sqrt(1 + 8 * x) + 1) // 2
    if val * (val - 1) != 2 * x:
        raise ValueError("Invalid number of lower triangular elements")
    return val


def fill_triangular(vec: torch.Tensor, upper: bool = False) -> torch.Tensor:
    """
    Args:
        vec: A tensor of shape (..., n(n-1)/2)
        upper: whether to fill the upper or lower triangle
    Returns:
        An array of shape (..., n, n), where the strictly upper (lower) triangle is filled from vec
        with zeros elsewhere
    """
    num_nodes = num_lower_tri_elements_to_n(vec.shape[-1])
    idxs = torch.triu_indices(num_nodes, num_nodes, offset=1, device=vec.device)
    output = torch.zeros(vec.shape[:-1] + (num_nodes, num_nodes), device=vec.device)
    output[..., idxs[0, :], idxs[1, :]] = vec
    return output if upper else output.transpose(-1, -2)


"""
Construct the matrix logit(pᵢⱼ).


See the class docstring.
We use the following derivation
    logit(pᵢⱼ) = - log(pᵢⱼ⁻¹ - 1)
                = - log((1 + exp(-γᵢⱼ))(1 + exp(-θᵢⱼ)) - 1)
                = - log(exp(-γᵢⱼ) + exp(-θᵢⱼ) + exp(-θᵢⱼ - γᵢⱼ))
                = - logsumexp([-γᵢⱼ, -θᵢⱼ, -θᵢⱼ - γᵢⱼ])
"""
logits_exist = sem_module.adjacency_module.adjacency_distribution.logits_exist
logits_orient = sem_module.adjacency_module.adjacency_distribution.logits_orient

neg_theta = fill_triangular(logits_orient, upper=True) - fill_triangular(logits_orient, upper=False)
logits_matrix =  -torch.logsumexp(
                    torch.stack([-logits_exist, neg_theta, neg_theta - logits_exist], dim=-1), dim=-1
                )


logits_matrix = logits_matrix.cpu().detach().numpy()
# take sigmoid of logits_matrix to get the probability matrix
prob_matrix = 1 / (1 + np.exp(-logits_matrix))

print(prob_matrix.shape)

In [ ]:
# sem_module.adjacency_module.adjacency_distribution.
logits_matrix = sem_module()._adjacency_dist.dist._get_independent_bernoulli_logits().cpu().detach().numpy()
prob_matrix = 1 / (1 + np.exp(-logits_matrix))
prob_matrix

In [ ]:
# round the probability matrix to 2 decimal places to get the adjacency matrix
adj_matrix = np.round(prob_matrix, 2)

adj_matrix_avg[adj_matrix_avg < 0.5] = 0 # Only keep edges with weight >= 0.5

graph = nx.from_numpy_array(adj_matrix_avg, create_using=nx.DiGraph)
graph = nx.relabel_nodes(graph, dict(enumerate(data_module.dataset_train.keys())))

labels = {node: i for i, node in enumerate(graph.nodes)}

layout = nx.circular_layout(graph)
# layout = nx.nx_agraph.graphviz_layout(graph, prog="dot")

fig, axis = plt.subplots(1, 1, figsize=(8, 8))

for node, i in labels.items():
    axis.scatter(layout[node][0], layout[node][1], label=f"{i}: {node}")

# add legend to right side of the plot outside the plot area
axis.legend(loc="center left", bbox_to_anchor=(1.05, 0.5))

nx.draw_networkx(graph, pos=layout, with_labels=True, arrows=True, labels=labels, ax=axis)  

# Get weights of each edge and assign to labels
labels = nx.get_edge_attributes(graph, "weight")

# Draw edge labels using layout and list of labels
nx.draw_networkx_edge_labels(graph, pos=layout, edge_labels=labels)

### Estimate Prediction Scores from the learned SEM
Use appropriate evaluation metrics based on whether the variable is binary or continuous

In [ ]:
# Get predictions of each variable from the model
test_data=data_module._dataset_train
prediction = sem.func(test_data, sem.graph)
prediction = sem.noise_dist(prediction).mode

#### Binary Variable

In [ ]:
from sklearn.metrics import classification_report

# Assuming y_true and y_pred are the ground truth and predicted labels respectively
y_true = test_data["Discount"].detach().numpy()
y_pred = prediction["Discount"].detach().numpy()

# Compute classification report
report = classification_report(y_true, y_pred)

# Print the classification report
print("\nClassification report:")
print(report)

#### Continuous Variable

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming y_true and y_pred are the ground truth and predicted labels respectively
y_true = test_data["Revenue"].detach().numpy()
y_pred = prediction["Revenue"].detach().numpy()

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse) # or use mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print('Mean Absolute Error (MAE): {}'.format(mae))
print('Mean Squared Error (MSE): {}'.format(mse))
print('Root Mean Squared Error (RMSE): {}'.format(rmse))
print('R² Score: {}'.format(r2))

In [ ]:
# Plot the predicted vs ground truth values for the continuous variable 
# Create a new figure
plt.figure()
# Scatter plot of y_true vs y_pred
plt.scatter(y_true, y_pred, color='blue', alpha=0.25, label='y_true vs y_pred')
# Line plot for y=x line
x = np.linspace(min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max()), 100)
plt.plot(x, x, '--', color='red', label='y=x line')

# Labels and legend
plt.xlabel('Ground Truth')
plt.ylabel('Predicted')
plt.title("Revenue")
plt.legend()

# Show the plot
plt.show()